# 01 · Preparación de datos (QC + splits + tasks.json)

In [ ]:
from pathlib import Path
import pandas as pd, numpy as np, json
ROOT=Path.cwd().parent; RAW=ROOT/'data'/'raw'/'udacity'; PROC=ROOT/'data'/'processed'
RUNS=['circuito1','circuito2']
def _fix(p): p=str(p).strip().strip('"').strip("'").replace('\\','/'); return p
def load_csv(csv_path):
    df=pd.read_csv(csv_path, header=None, names=['center','left','right','steering','throttle','brake','speed'])
    for cam in ['center','left','right']:
        df[cam]=df[cam].map(_fix)
        df[cam]=df[cam].apply(lambda s: s[s.lower().rfind('img/'):] if 'img/' in s.lower() else s)
    return df
def stratified(df,bins=21,train=0.7,val=0.15,seed=42):
    s=df['steering'].astype(float); lo=min(float(s.min()),-1.0); hi=max(float(s.max()),1.0)
    edges=np.linspace(lo,hi,bins); labels=pd.cut(s,bins=edges,include_lowest=True,labels=False)
    df=df.copy(); df['_b']=labels; rng=np.random.default_rng(seed); parts=[]
    for b in sorted(df['_b'].dropna().unique()):
        g=df[df['_b']==b]; idx=np.arange(len(g)); rng.shuffle(idx)
        n=len(idx); ntr=int(round(n*train)); nva=int(round(n*val))
        parts.append((g.iloc[idx[:ntr]], g.iloc[idx[ntr:ntr+nva]], g.iloc[idx[ntr+nva:]]))
    tr=pd.concat([a for a,_,_ in parts], ignore_index=True)
    va=pd.concat([b for _,b,_ in parts], ignore_index=True)
    te=pd.concat([c for _,_,c in parts], ignore_index=True)
    for p in [tr,va,te]: p.drop(columns=['_b'], inplace=True, errors='ignore')
    return tr,va,te
PROC.mkdir(parents=True, exist_ok=True); manif=[]
for run in RUNS:
    base=RAW/run; csv=base/'driving_log.csv'; assert csv.exists(), f'No {csv}'
    df=load_csv(csv)
    for cam in ['center','left','right']: df=df[(base/df[cam]).apply(lambda p: p.exists())]
    out=PROC/run; out.mkdir(parents=True, exist_ok=True); df.to_csv(out/'canonical.csv', index=False)
    tr,va,te=stratified(df); tr.to_csv(out/'train.csv', index=False); va.to_csv(out/'val.csv', index=False); te.to_csv(out/'test.csv', index=False)
    manif.append({'run':run,'paths':{'train':str(out/'train.csv'),'val':str(out/'val.csv'),'test':str(out/'test.csv')}})
with open(PROC/'tasks.json','w',encoding='utf-8') as f: json.dump({'tasks_order':RUNS,'splits':{m['run']:m['paths'] for m in manif}}, f, indent=2, ensure_ascii=False)
print('OK:', PROC/'tasks.json')